In [3]:
!python -m venv zenml_env
!source zenml_env/bin/activate 
!pip install zenml scikit-learn pandas
!zenml init

'source' is not recognized as an internal or external command,
operable program or batch file.


Initializing the ZenML global configuration version to 0.80.2
⠋ Initializing ZenML repository at c:\Users\PRAVEEN 
ANANTH\OneDrive\Desktop\Biriyani\Training\Level 20.

⠋ Initializing ZenML repository at c:\Users\PRAVEEN 
ANANTH\OneDrive\Desktop\Biriyani\Training\Level 20.

⠹ Initializing ZenML repository at c:\Users\PRAVEEN 
ANANTH\OneDrive\Desktop\Biriyani\Training\Level 20.

⠸ Initializing ZenML repository at c:\Users\PRAVEEN 
ANANTH\OneDrive\Desktop\Biriyani\Training\Level 20.

⠼ Initializing ZenML repository at c:\Users\PRAVEEN 
ANANTH\OneDrive\Desktop\Biriyani\Training\Level 20.

⠴ Initializing ZenML repository at c:\Users\PRAVEEN 
ANANTH\OneDrive\Desktop\Biriyani\Training\Level 20.

⠦ Initializing ZenML repository at c:\Users\PRAVEEN 
ANANTH\OneDrive\Desktop\Biriyani\Training\Level 20.

⠧ Initializing ZenML repository at c:\Users\PRAVEEN 
ANANTH\OneDrive\Desktop\Biriyani\Training\Level 20.

⠇ Initializing ZenML repository at c:\Users\PRAVEEN 
ANANTH\OneDrive\Desktop\Biriyani\Trai

In [6]:
!pip install zenml scikit-learn pandas


In [22]:
from zenml import pipeline, step
from pydantic import BaseModel
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from typing import Tuple


class IngestDataParameters(BaseModel):
    dataset_name: str = "diabetes"

@step
def data_ingest(params: IngestDataParameters) -> pd.DataFrame:
    if params.dataset_name == "diabetes":
        data = load_diabetes()
        df = pd.DataFrame(data.data, columns=data.feature_names)
        df["target"] = data.target
    return df


@step
def data_cleaning(df: pd.DataFrame) -> pd.DataFrame:
    return df.dropna()


@step
def model_train(df: pd.DataFrame) -> Tuple[LinearRegression, pd.DataFrame, pd.Series]:
    X = df.drop("target", axis=1)
    y = df["target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    return model, X_test, y_test


@step
def model_evaluate(model: LinearRegression, X_test: pd.DataFrame, y_test: pd.Series) -> Tuple[float, float]:
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MSE: {mse:.2f}")
    print(f"R² Score: {r2:.2f}")
    return mse, r2

@pipeline
def diabetes_pipeline(ingest_data_params: IngestDataParameters):
    df = data_ingest(params=ingest_data_params)
    cleaned_df = data_cleaning(df=df)
    model, X_test, y_test = model_train(df=cleaned_df)
    model_evaluate(model=model, X_test=X_test, y_test=y_test)



In [ ]:
if __name__ == "__main__":
    diabetes_pipeline(ingest_data_params=IngestDataParameters())

Initiating a new run for the pipeline: diabetes_pipeline.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Using cached version of step data_ingest.
Using cached version of step data_cleaning.
Using cached version of step model_train.
Using cached version of step model_evaluate.
All steps of the pipeline run were cached.
